# Train/Test Split
Tasks solved by the notebook:
* Check of offical test set (Christian)
* Removal of test datasests from training set
* Adding some columns to official test set for our convinience
* Storing of official test set in parquet format
* Stroing of an unbalanced train set
* Storing of an balanced train set

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import fastparquet as fp
import matplotlib.pyplot as plt
import sklearn.model_selection as skms
import src.helper.helper as hlp
import src.helper.const as const
import os
import re

In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs

# Configuration

In [2]:
# Config
SEED = const.SEED
DATA_DIR_INTERIM = const.DATA_DIR_INTERIM
DATA_DIR_RAW = const.DATA_DIR_RAW
POSTER_DIR = const.POSTER_DIR

VERSION = "v3"

# Input files
FN_DF_TRAIN_EVAL_IN = DATA_DIR_INTERIM + f"df_multilabel_{VERSION}.gzip"
FN_DF_TEST_IN = DATA_DIR_RAW + f"movies_test.csv" #  Datasets for official testing
FN_DF_TRAIN_BAL_IN = DATA_DIR_INTERIM + f"df_train-split_v1.gzip"  # former processed balanced train set
FN_DF_EVAL_BAL_IN = DATA_DIR_INTERIM + f"df_test-split_v1.gzip"  # former processed balanced eval set

# Filnames of unbalanced train, eval sets
FN_DF_TRAIN_UNBAL_OUT = DATA_DIR_INTERIM + f"df_train_unbalanced_{VERSION}.gzip"
# not used anymore: FN_DF_EVAL_UNBAL_OUT = DATA_DIR_INTERIM + f"df_eval_unbalanced_{VERSION}.gzip"

# Filnames of balanced train, eval sets
FN_DF_TRAIN_BAL_OUT = DATA_DIR_INTERIM + f"df_train_balanced_{VERSION}.gzip"
# not used anymore: FN_DF_EVAL_BAL_OUT = DATA_DIR_INTERIM + f"df_eval_balanced_{VERSION}.gzip"

# Filnames of train, eval set using the balance of the official testset
# not implemented yet: FN_DF_TRAIN_TESTBAL_OUT = DATA_DIR_INTERIM + f"df_train_tstbalance_{VERSION}.gzip"
# not implemented yet: FN_DF_EVAL_TESTBAL_OUT = DATA_DIR_INTERIM + f"df_eval_tstbalance_{VERSION}.gzip"

# Filname of official testset
FN_DF_TEST_TESTBAL_OUT = DATA_DIR_INTERIM + f"df_test_{VERSION}.gzip"

# Initialization

In [3]:
# Init
pd.set_option('display.max_colwidth', None)
GENRE_COLS = const.GENRE_OHE_COLS

# Helper functions

In [4]:
# Helper functions
def plot_gen_distribution(df: pd.DataFrame, cols=GENRE_COLS, topic: str = None, show_corrmatrix = False):
    if topic == None:
        print(f"Genre distribution for {len(df)} datasets")
    else:
        print(f"Genre distribution for {len(df)} {topic} datasets")

    # Plot distribution values
    for col in cols:
        print(f"{col}: {df[col].value_counts()[1]/len(df):>{25 - len(col)}.4f}")

    # Plot correlation matrix
    if show_corrmatrix:
        print()
        hlp.plot_correlations(df, num_cols=GENRE_COLS, figsize=(18, 11))

# Load files

In [5]:
# Load training datasets incl. evaluation sets
df = pd.read_parquet(FN_DF_TRAIN_EVAL_IN)

# Load official test sets
df_test = pd.read_csv(FN_DF_TEST_IN, sep=";")

In [6]:
display(df.shape)
display(df.head(3))

(274855, 32)

,id,original_title,release_date,popularity,adult,video,poster_url,poster_exists,filename,genre_id,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,649583,The Grass Ceiling,2019-11-16,0.600,False,False,https://www.themoviedb.org/t/p/w500//koxKUUCOE15D6cd2IW1CBnqfJu1.jpg,True,koxKUUCOE15D6cd2IW1CBnqfJu1.jpg,[99],...,0,0,0,0,0,0,0,0,0,0
1,649584,Alleycats,2017-11-15,1.029,False,False,https://www.themoviedb.org/t/p/w500//e7u2toV4ZqKszvVY9I3bUmJMgma.jpg,True,e7u2toV4ZqKszvVY9I3bUmJMgma.jpg,"[28, 16]",...,0,0,0,0,0,0,0,0,0,0
2,649586,Качалка,2020-06-17,0.600,False,False,https://www.themoviedb.org/t/p/w500//b3w7FK12ahYSpAWqzsZJip1WYNf.jpg,True,b3w7FK12ahYSpAWqzsZJip1WYNf.jpg,[99],...,0,0,0,0,0,0,0,0,0,0


In [7]:
display(df_test.shape)
display(df_test.head(3))

(1000, 23)

,id,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,14813,Mickey's Christmas Carol,/6QH10DqZ0feBW2phvrtoLqT1s0k.jpg,['Family' 'Animation'],0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7177,Fire from Below,/9eecXogqTUMpsDMNZD8cCA5btMG.jpg,['Action' 'Adventure' 'Science Fiction' 'Thriller'],1,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,11292,Seventeen - Mädchen sind die besseren Jungs,/oUgbUWfPlAQhKLPnX6N9gAjKkvJ.jpg,['Comedy'],0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Check quality of official test set

## Show missing values

In [8]:
df_test.isna().sum()

id                 0
original_title     0
poster_path        0
genre_names        0
Action             0
Adventure          0
Animation          0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
History            0
Horror             0
Music              0
Mystery            0
Romance            0
Science Fiction    0
TV Movie           0
Thriller           0
War                0
Western            0
dtype: int64

## Check genre_id
Datasets without a "genre_name" should be recognized too!

In [9]:
df_test.loc[(df_test.genre_names=='') | (df_test.genre_names=='[]')]

,id,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
126,18715,Giai Cuu Than Chet,/oxEB2GCRqoO8VczNglXZI2Mr54W.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176,19148,Roam,/xKgab7KpajrCMUfksG0Iio6h8S4.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
371,17528,UFC 24: First Defense,/bKyh9sdoADNVOa2bJi3RB80QeQP.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
381,19078,UFC 101: Declaration,/o1ayuAaAhvoG1JQKZJmtidAbl96.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
413,13084,UFC 81: Breaking Point,/vWUV0HmoOlAY9rXCCjsDjF5sJeG.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
418,17840,Pride 32: The Real Deal,/duv9mTBXhUdIMgMBetNPxaKCQr3.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503,14775,Erskineville Kings,/hiGVv0vmanLnH8dXGMZjjNSkacO.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506,18314,UFC 98: Evans vs. Machida,/kTxRFU9awLpWVHT3fX0aVjZbBjY.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
508,13047,UFC 87: Seek and Destroy,/dg7Z3PQn8NzPQcQEwQSfF5DLLL2.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
532,13623,UFC 91: Couture vs. Lesnar,/zYHdUtNaU2XibIXeAoBlOBpECKO.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Check availability of posters for the official test set

In [10]:
# Check it
def check_image_exists(df):
    '''Checks the presence of a filename and fills a column regarding
    '''
    l_files = os.listdir(POSTER_DIR)
    l_check_res = []
    for idx, row in df.iterrows():
        pfname = re.findall("\w+.jpg", row.poster_path)
        l_check_res.append(pfname[0] in l_files)
    return pd.DataFrame(l_check_res, columns=["poster_exists"])

# Write result to column "poster_exists"
df_res = hlp.parallelize_dataframe(df_test, check_image_exists)
df_test["poster_exists"] = df_res["poster_exists"].to_list()
if df_test.poster_exists.sum() != len(df_test):
    print("Datasets to be removed, because of missing poster or wrong poster path:")
    display(df_test.loc[(df_test.poster_exists == False), ["id", "poster_path", "poster_exists"]].sort_values("id"))

Datasets to be removed, because of missing poster or wrong poster path:


,id,poster_path,poster_exists
965,242,/1hdm3Axw9LjITbApvAXBbqO58zE.jpg,False
690,8900,/mpYcisTwGJDnmlmjRrvySGcmbkX.jpg,False
823,10538,/4VBN8pQxGHjeZWcNv1V1xSw0OKC.jpg,False
575,12592,/3i8vNc8tJPsRFsEKRDnwrQ6KDfb.jpg,False
196,13757,/gDTl4hziGqLbvCtt00ac2RdPfVe.jpg,False
499,18125,/injKq5aeCiN33zuusXqV0Ybs75y.jpg,False
392,18628,/x5q0grSPmXPlWYrO9Qr9G9tnPfQ.jpg,False
322,18944,/tQvYyXnPxwEEmeqmk4BDPaQWM96.jpg,False


In [11]:
# Check in df
print("Poster path and availability within train/eval set:")
mask = df.id.isin(df_test[df_test.poster_exists == False].id)
df.loc[mask, ["id", "filename", "poster_exists"]].sort_values("id")

Poster path and availability within train/eval set:


,id,filename,poster_exists
54971,242,hOlI8nLlVcH6JKrrMGGqYCQewWz.jpg,True
58639,8900,lQVngPh7jXN05ZExED9RmVUurr2.jpg,True
59939,10538,41Qz2IyVhmSJoCi1UhJHyDqEugL.jpg,True
61503,12592,tDzoBcMGMS8R6Ltg9lIEhuZqG7N.jpg,True
62221,13757,aLij9PB33f4PscrJqin03TlnZtv.jpg,True
176442,18125,bNofq5QmwFeFTILeUOzwG9hPe1X.jpg,True
176761,18628,zXkmmx9YD2N15YhcaWTG24NCPmc.jpg,True
176969,18944,4vm06pTpw2yfjaMV1j2weXkTJYL.jpg,True


## Show test dataset not in train/eval
For informal purpose only!

In [12]:
display(df_test[~df_test.id.isin(df.id.to_list())])

,id,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,poster_exists
126,18715,Giai Cuu Than Chet,/oxEB2GCRqoO8VczNglXZI2Mr54W.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
176,19148,Roam,/xKgab7KpajrCMUfksG0Iio6h8S4.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
371,17528,UFC 24: First Defense,/bKyh9sdoADNVOa2bJi3RB80QeQP.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
381,19078,UFC 101: Declaration,/o1ayuAaAhvoG1JQKZJmtidAbl96.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
413,13084,UFC 81: Breaking Point,/vWUV0HmoOlAY9rXCCjsDjF5sJeG.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
418,17840,Pride 32: The Real Deal,/duv9mTBXhUdIMgMBetNPxaKCQr3.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
503,14775,Erskineville Kings,/hiGVv0vmanLnH8dXGMZjjNSkacO.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
506,18314,UFC 98: Evans vs. Machida,/kTxRFU9awLpWVHT3fX0aVjZbBjY.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
508,13047,UFC 87: Seek and Destroy,/dg7Z3PQn8NzPQcQEwQSfF5DLLL2.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
532,13623,UFC 91: Couture vs. Lesnar,/zYHdUtNaU2XibIXeAoBlOBpECKO.jpg,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


# Correct official test set

## Replace wrong poster_path

In [13]:
l_pp = []
if df_test.poster_exists.sum() != len(df_test):
    for i, r in df_test.loc[df_test.poster_exists == False].iterrows():
        l_pp.append(df[df.id == r.id].filename.values[0])
df_test.loc[df_test.poster_exists == False, "poster_path"] = l_pp

## Add new columns for compatibilty to train set

In [14]:
# Convert some column names
df_test["filename"] = df_test["poster_path"].map(lambda x: re.findall("\w+.jpg", x))
df_test["genre_ids2"] = df_test["genre_names"].map(lambda x: re.sub(" ", ",", re.sub("\'", "", x)))
df_test["genre_ids2_list"] = df_test["genre_ids2"].map(lambda x: np.array([re.sub("\[|\]", "", e) for e in re.split(",", x)]))

# Plot distribution
plot_gen_distribution(df_test, topic="Official test")

Genre distribution for 1000 Official test datasets
Action:              0.1800
Adventure:           0.1160
Animation:           0.0650
Comedy:              0.3560
Crime:               0.1330
Documentary:         0.0410
Drama:               0.4330
Family:              0.1010
Fantasy:             0.0840
History:             0.0480
Horror:              0.1070
Music:               0.0510
Mystery:             0.0660
Romance:             0.1610
Science Fiction:     0.1010
TV Movie:            0.0190
Thriller:            0.2230
War:                 0.0320
Western:             0.0140


# Save official test set in parquet format

In [15]:
df_test.to_parquet(FN_DF_TEST_TESTBAL_OUT)

# Remove official test sets from training/evaluation set

In [16]:
# Remove official test datasets from train-evaluation set
ids_2_remove = set(df_test.id.tolist())
lb = len(df)
print(f"Number of rows before removal of {len(ids_2_remove)} test-sets: {lb}")
df = df[~df.id.isin(ids_2_remove)]
la = len(df)
print(f"Number of rows after removal of {len(ids_2_remove)} test-sets:  {la}")
if (lb-la) == len(ids_2_remove):
    print(f"All official test datasets removed from train/eval set!")
else:
    print(f"{lb - la} official test datasets removed from train/eval set!")

Number of rows before removal of 1000 test-sets: 274855
Number of rows after removal of 1000 test-sets:  273876
979 official test datasets removed from train/eval set!


In [17]:
# Show distribution of datasets
plot_gen_distribution(df)

Genre distribution for 273876 datasets
Action:              0.0893
Adventure:           0.0479
Animation:           0.0690
Comedy:              0.2430
Crime:               0.0624
Documentary:         0.1847
Drama:               0.3680
Family:              0.0528
Fantasy:             0.0361
History:             0.0261
Horror:              0.0786
Music:               0.0731
Mystery:             0.0338
Romance:             0.0977
Science Fiction:     0.0379
TV Movie:            0.0348
Thriller:            0.0873
War:                 0.0199
Western:             0.0166


# Create unbalanced train/eval set

In [18]:
"""Not used anymore
# Train, evaluation split
df_train, df_eval = skms.train_test_split(df, test_size=0.2, random_state=SEED)
plot_gen_distribution(df_train, topic="Train")
print()
plot_gen_distribution(df_eval, topic="Eval")
"""
df_train = df.copy()
plot_gen_distribution(df_train, topic="Train unbalanced")

Genre distribution for 273876 Train unbalanced datasets
Action:              0.0893
Adventure:           0.0479
Animation:           0.0690
Comedy:              0.2430
Crime:               0.0624
Documentary:         0.1847
Drama:               0.3680
Family:              0.0528
Fantasy:             0.0361
History:             0.0261
Horror:              0.0786
Music:               0.0731
Mystery:             0.0338
Romance:             0.0977
Science Fiction:     0.0379
TV Movie:            0.0348
Thriller:            0.0873
War:                 0.0199
Western:             0.0166


## Save unbalanced sets to file

In [19]:
# Write splits to file
df_train.to_parquet(FN_DF_TRAIN_UNBAL_OUT, compression='gzip')
# not used anymore: df_eval.to_parquet(FN_DF_EVAL_UNBAL_OUT, compression='gzip')

# Create balanced train/eval sets 
Using the existing one

In [37]:
# Load files
df_train = pd.read_parquet(FN_DF_TRAIN_BAL_IN)
df_eval = pd.read_parquet(FN_DF_EVAL_BAL_IN)
print(df_train.shape, df_eval.shape)

# Merge files
df_train = pd.concat([df_train, df_eval])
print(df_train.shape)

(11764, 26) (3677, 26)
(15441, 26)


In [38]:
display(df_train.loc[df_train.id == 105])

,adult,id,original_title,popularity,video,url,poster_url,x0_Action,x0_Adventure,x0_Animation,...,x0_History,x0_Horror,x0_Music,x0_Mystery,x0_Romance,x0_Science Fiction,x0_TV Movie,x0_Thriller,x0_War,x0_Western
14474,False,105,Back to the Future,38.725,False,https://www.themoviedb.org/movie/105,https://www.themoviedb.org/t/p/w500//xlBivetfrtF84Yx0zISShnNtHYe.jpg,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [39]:
# Correct old fashioned column names
df_train = df_train.rename(columns={
    'x0_Action': 'Action',
    'x0_Animation': 'Animation',
    'x0_Adventure': 'Adventure',
    'x0_Comedy': 'Comedy',
    'x0_Crime': 'Crime',
    'x0_Documentary': 'Documentary',
    'x0_Drama': 'Drama',
    'x0_Family': 'Family',
    'x0_Fantasy': 'Fantasy',
    'x0_History': 'History',
    'x0_Horror': 'Horror',
    'x0_Music': 'Music',
    'x0_Mystery': 'Mystery',
    'x0_Romance': 'Romance',
    'x0_Science Fiction': 'Science Fiction',
    'x0_TV Movie': 'TV Movie',
    'x0_Thriller': 'Thriller',
    'x0_War': 'War',
    'x0_Western': 'Western'})

# Load missing column names
df_train = df_train.merge(df[["id", "filename", "genre_ids2", "genre_ids2_list"]], how='left', on="id")

## => Maybe REDESIGN necessary
Because most of the Actions not right filled

In [40]:
display(df_train.loc[df_train.id == 105, ["id", "genre_ids2_list"] + const.GENRE_OHE_COLS])
display(df.loc[df.id == 105, ["id", "genre_ids2_list"] + const.GENRE_OHE_COLS])

,id,genre_ids2_list,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
15437,105,"[Adventure, Comedy, Science Fiction, Family]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


,id,genre_ids2_list,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
15437,105,"[Adventure, Comedy, Science Fiction, Family]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [41]:
# Remove official test datasets
ids_2_remove = set(df_test.id.tolist())
lb = len(df_train)
print(f"Number of rows before removal of {len(ids_2_remove)} test-sets: {lb}")
df = df_train[~df_train.id.isin(ids_2_remove)]
la = len(df_train)
print(f"Number of rows after removal of {len(ids_2_remove)} test-sets:  {la}")
if (lb-la) == len(ids_2_remove):
    print(f"All official test datasets removed from train set!")
else:
    print(f"{lb - la} official test datasets removed from train/eval set!")
print()

# Plot distribution
plot_gen_distribution(df_train, topic="Train balanced")

Number of rows before removal of 1000 test-sets: 15441
Number of rows after removal of 1000 test-sets:  15441
0 official test datasets removed from train/eval set!

Genre distribution for 15441 Train balanced datasets
Action:              0.0539
Adventure:           0.0547
Animation:           0.0544
Comedy:              0.0539
Crime:               0.0528
Documentary:         0.0543
Drama:               0.0547
Family:              0.0542
Fantasy:             0.0547
History:             0.0547
Horror:              0.0548
Music:               0.0556
Mystery:             0.0552
Romance:             0.0540
Science Fiction:     0.0547
TV Movie:            0.0539
Thriller:            0.0543
War:                 0.0551
Western:             0.0532


## Save balanced sets to file

In [42]:
# Write splits to file
df_train.to_parquet(FN_DF_TRAIN_BAL_OUT, compression='gzip')
# not used anymore: df_eval.to_parquet(FN_DF_EVAL_BAL_OUT, compression='gzip')

# Create train/eval sets regarding official test set balance

In [25]:
# Not iimplemented yet

## Save sets to file

In [26]:
# Write splits to file
# df_train_.to_parquet(FN_DF_TRAIN_TESTBAL_OUT, compression='gzip')